# LIBRARY IMPORTS

In [58]:
%run importing.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LOAD DATA

In [59]:
df = pd.read_csv('train.csv')

# EDA

In [61]:
X, y, ohe = eda(df)

# PREPROCESSING

In [62]:
pX, py = process(X, y)

# SPLIT DATA

In [63]:
split_size = 0.25
X_train, X_val, y_train, y_val = train_test_split(pX, py, test_size=split_size, random_state=42)
X_train.shape, X_val.shape

((1095, 305), (365, 305))

# MODELING

In [64]:
lin_params = {'fit_intercept': [0,1], 'normalize': [0,1]}
lin_grid = GridSearchCV(LinearRegression(), lin_params, cv=5, n_jobs=-1).fit(X_train, y_train)
lin = LinearRegression(**lin_grid.best_params_).fit(X_train, y_train)

In [66]:
knn_params = {'n_neighbors': range(1,10), 
              'weights': ['uniform', 'distance'], 
              'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}
knn_grid = GridSearchCV(KNeighborsRegressor(), knn_params, cv=5, n_jobs=-1).fit(X_train, y_train)
knn = KNeighborsRegressor(**knn_grid.best_params_).fit(X_train, y_train)

In [69]:
rfr_params = {'n_estimators': range(100, 501, 100), 
              'max_depth': range(1,10)}
rfr_grid = GridSearchCV(RandomForestRegressor(), rfr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
rfr = RandomForestRegressor(**rfr_grid.best_params_).fit(X_train, y_train)

In [71]:
abr_params = {'n_estimators': range(100, 501, 100), 
              'learning_rate': np.logspace(-4, 0, 5)}
abr_grid = GridSearchCV(AdaBoostRegressor(), abr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
abr = AdaBoostRegressor(**abr_grid.best_params_).fit(X_train, y_train)

In [73]:
gbr_params = {'n_estimators': range(100, 501, 100), 
              'learning_rate': np.logspace(-4, 0, 5),
              'max_depth': range(1, 10)}
gbr_grid = GridSearchCV(GradientBoostingRegressor(), gbr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
gbr = GradientBoostingRegressor(**gbr_grid.best_params_).fit(X_train, y_train)

In [75]:
cbr = CatBoostRegressor(verbose=False, iterations=2000).fit(X_train, y_train)

In [76]:
print('lin:', mean_squared_error(y_val, lin.predict(X_val)) ** 0.5)
print('knn:', mean_squared_error(y_val, knn.predict(X_val)) ** 0.5)
print('rfr:', mean_squared_error(y_val, rfr.predict(X_val)) ** 0.5)
print('abr:', mean_squared_error(y_val, abr.predict(X_val)) ** 0.5)
print('gbr:', mean_squared_error(y_val, gbr.predict(X_val)) ** 0.5)
print('cbr:', mean_squared_error(y_val, cbr.predict(X_val)) ** 0.5)

lin: 21851865856.929813
knn: 0.20387896456549873
rfr: 0.1426807457980089
abr: 0.17560001903029365
gbr: 0.12884514426460536
cbr: 0.1928385460071715


In [44]:
# model = Sequential()
# model.add(Dense(units=2096, activation='relu', input_shape=(265,)))
# model.add(Dropout(0.25))
# model.add(Dense(units=1024, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(units=1, activation='linear'))
# opt = optimizers.SGD()
# model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse])
# model.summary()

In [45]:
# history = model.fit(X_train, y_train, epochs=201, batch_size=25, verbose=0, validation_split=0.3,
#                     shuffle=True, callbacks=[Logger(50)])

In [10]:
model = GradientBoostingRegressor(n_estimators=400, max_depth=3).fit(pX, py)

In [11]:
df_kaggle = pd.read_csv('test.csv')
X_kaggle, _ = eda(df_kaggle, False)
X_kaggle, _ = process(X_kaggle)

In [12]:
dict_kaggle = {'Id' : df_kaggle['Id'], 'SalePrice': np.exp(model.predict(X_kaggle))}
pd.DataFrame(dict_kaggle).to_csv('predictions.csv', index=False)